In [1]:
import geopandas as gpd
import requests
from IPython.display import display

# URL do GeoJSON
url = "https://raw.githubusercontent.com/tbrugz/geodata-br/master/geojson/geojs-31-mun.json"

# 1. Baixar e carregar os dados
try:
    print("Baixando dados...")
    gdf = gpd.read_file(url)
    
    # 2. Verificar e definir o CRS original (assumindo WGS84)
    if gdf.crs is None:
        print("\n⚠️ CRS não definido. Assumindo EPSG:4326 (WGS84)")
        gdf = gdf.set_crs('EPSG:4326')
    
    print("\n✅ Dados carregados com sucesso!")
    print(f"CRS atual: {gdf.crs}")
    print(f"Número de municípios: {len(gdf)}")
    display(gdf.head(2))
    
except Exception as e:
    print(f"❌ Erro ao carregar os dados: {e}")
    gdf = None

# 3. Conversão para EPSG:31983 e cálculo da área
if gdf is not None:
    try:
        print("\nConvertendo para EPSG:31983...")
        gdf_31983 = gdf.to_crs('EPSG:31983')
        
        # Calcular área em km² (31983 já está em metros, então m² → km²)
        gdf_31983['area_km2'] = gdf_31983.geometry.area / 1_000_000
        gdf_31983['area_km2'] = gdf_31983['area_km2'].round(2)  # Arredondar para 2 decimais
        
        print("\n✅ Conversão concluída e áreas calculadas!")
        print(f"Novo CRS: {gdf_31983.crs}")
        print("Exemplo dos dados com área:")
        display(gdf_31983[['name', 'area_km2']].head())
        
        # Salvar GeoJSON
        geojson_file = "municipios_mg_31983.geojson"
        gdf_31983.to_file(geojson_file, driver='GeoJSON')
        print(f"\nGeoJSON salvo como: {geojson_file}")
        
        # Salvar Excel (sem a coluna de geometria)
        excel_file = "municipios_mg_31983.xlsx"
        gdf_31983.drop(columns='geometry').to_excel(excel_file, index=False)
        print(f"Excel salvo como: {excel_file}")
        
        # Mostrar estatísticas das áreas
        print("\n📊 Estatísticas das áreas municipais (km²):")
        print(f"Média: {gdf_31983['area_km2'].mean():.2f} km²")
        print(f"Maior: {gdf_31983['area_km2'].max():.2f} km² ({gdf_31983.loc[gdf_31983['area_km2'].idxmax(), 'name']})")
        print(f"Menor: {gdf_31983['area_km2'].min():.2f} km² ({gdf_31983.loc[gdf_31983['area_km2'].idxmin(), 'name']})")
        
    except Exception as e:
        print(f"❌ Erro na conversão: {e}")

Baixando dados...

✅ Dados carregados com sucesso!
CRS atual: EPSG:4326
Número de municípios: 853


,id,name,description,geometry
0,3100104,Abadia dos Dourados,Abadia dos Dourados,"POLYGON ((-47.4441 -18.15847, -47.43543 -18.16..."
1,3100203,Abaeté,Abaeté,"POLYGON ((-45.182 -18.88757, -45.17314 -18.889..."



Convertendo para EPSG:31983...

✅ Conversão concluída e áreas calculadas!
Novo CRS: EPSG:31983
Exemplo dos dados com área:


,name,area_km2
0,Abadia dos Dourados,895.12
1,Abaeté,1814.50
2,Abre Campo,472.05
3,Acaiaca,100.72
4,Açucena,812.88



GeoJSON salvo como: municipios_mg_31983.geojson
Excel salvo como: municipios_mg_31983.xlsx

📊 Estatísticas das áreas municipais (km²):
Média: 688.06 km²
Maior: 10709.34 km² (João Pinheiro)
Menor: 2.51 km² (Santa Cruz de Minas)


In [2]:
# Adicionar esta opção após a conversão
if gdf is not None:
    # ... (código anterior)
    
    # Opção alternativa para Excel com geometria
    excel_with_geom = "municipios_mg_31983_com_geometria.xlsx"
    gdf_export = gdf_31983.copy()
    gdf_export['geometry_wkt'] = gdf_31983.geometry.apply(lambda x: x.wkt)
    gdf_export.to_excel(excel_with_geom, index=False)
    
    print(f"\nExcel com geometria salvo como: {excel_with_geom}")


Excel com geometria salvo como: municipios_mg_31983_com_geometria.xlsx


In [7]:
import geopandas as gpd
!pip install geopandas fiona openpyxl  # Adicionando openpyxl para suporte a Excel
import pandas as pd

# 1. Carregar os arquivos GPKG
gdf_ago = gpd.read_file("C:/Users/Dellg/tracbell/desmatamento_ago22.gpkg")
gdf_set = gpd.read_file("C:/Users/Dellg/tracbell/desmatamento_set22.gpkg")

# 2. Unir os datasets
gdf_unido = gpd.GeoDataFrame(pd.concat([gdf_ago, gdf_set], ignore_index=True))

# 3. Transformar para a projeção EPSG:31983 (SIRGAS 2000 / UTM zone 23S)
gdf_unido = gdf_unido.to_crs(epsg=31983)

# Visualizar as 5 primeiras linhas após transformação
print("\nPrimeiras 5 linhas após transformação de CRS:")
print(gdf_unido.head())

# 4. Salvar como GeoJSON
gdf_unido.to_file('focos-desmatamento-mg.geojson', driver='GeoJSON')

# 5. Exportar para Excel (arquivo .xlsx)
# Remover a coluna de geometria para o Excel (opcional, pois o Excel não suporta geometrias diretamente)
df_excel = gdf_unido.drop(columns=['geometry'])

# Salvar como Excel
df_excel.to_excel('focos-desmatamento-mg.xlsx', index=False, engine='openpyxl')

print("Arquivo GeoJSON criado com sucesso na projeção EPSG:31983!")
print("Arquivo Excel criado com sucesso: 'focos-desmatamento-mg.xlsx'")

ERROR: Invalid requirement: '#'



Primeiras 5 linhas após transformação de CRS:
   ogc_fid    id  Instância Data Inicial  Data Final   Cena Score  \
0      1.0  6726       9696   26/07/2022  05/08/2022  22KFC   0.9   
1      2.0  6727       9700   21/07/2022  26/07/2022  22KFD   0.9   
2      3.0  6728       9701   21/07/2022  26/07/2022  22KFD   0.9   
3      4.0  6729       9702   21/07/2022  05/08/2022  22KFD  0.93   
4      5.0  6730       9703   21/07/2022  05/08/2022  22KFD  0.91   

  Data Processamento    Bioma              NuFis Cia PM  \
0   08/08/2022 13:08  Cerrado  Triângulo Mineiro      5   
1   08/08/2022 13:08  Cerrado  Triângulo Mineiro      5   
2   08/08/2022 13:08  Cerrado  Triângulo Mineiro      5   
3   08/08/2022 13:08  Cerrado  Triângulo Mineiro      5   
4   08/08/2022 13:08  Cerrado  Triângulo Mineiro      5   

                                            geometry  
0  POLYGON ((65610.256 7779397.013, 65659.326 777...  
1  POLYGON ((31696.271 7842130.438, 31989.678 784...  
2  POLYGON ((50597